In [13]:
from functools import reduce


def factorial(n):
    """returns n!"""
    return reduce(lambda x, y: x * y, range(1, n + 1))

In [14]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [15]:
# Function objects have many attributes which can be introspected

dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [28]:
# Dict attribute __dict__ can be used to store user attributes like in Django


def upper_case_name(obj):
    return ("%s %s" % (obj.first_name, obj.last_name)).upper()

upper_case_name.short_description = 'Customer name'

In [18]:
upper_case_name.__dict__

{'short_description': 'Customer name'}

In [22]:
# List attributes of functions that don't exist in plain instances

class C: pass
obj = C()
def func(): pass

sorted(set(dir(func)) - set(dir(C)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

In [57]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [31]:
tag('br')   # Given single positional argument

'<br />'

In [32]:
tag('p', 'hello')

'<p>hello</p>'

In [33]:
tag('p', 'hello', 'world')

'<p>hello</p>\n<p>world</p>'

In [34]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [35]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [37]:
tag(content='testing', name='img')

'<img content="testing" />'

In [38]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
          'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

In [39]:
# Keyword only arguments are after positional
# If you don't want to support variable positional arguments
# But still want keyword-only arguments, put a * in signature


def f(a, *, b):
    return a, b

f(1, b=2)

(1, 2)

In [43]:
# Retrieving information about parameters


def hello(person, species='Homo Sapiens'):
    return 'Hello %s' % person

In [44]:
hello.__defaults__  # Defaults holds a tuple with the default values of positional and keyword parameters

('Homo Sapiens',)

In [45]:
hello.__kwdefaults__

In [46]:
# names of arguments are stored in __code__
hello.__code__.co_varnames

('person', 'species')

In [47]:
hello.__code__.co_argcount

2

In [49]:
# Lets introspect a clip function


def clip(text: str, max_len=80):
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # No spaces found
        end = len(text)
    return text[:end].rstrip()

In [50]:
clip.__defaults__

(80,)

In [51]:
clip.__code__

<code object clip at 0x7f5535e989c0, file "<ipython-input-49-0aabf0d6acb9>", line 4>

In [52]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [53]:
clip.__code__.co_argcount

2

In [54]:
# This information is really split between different tuples
# not easy or convenient to work with

from inspect import signature

sig = signature(clip)
sig

<Signature (text:str, max_len=80)>

In [55]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [56]:
sig.parameters

mappingproxy({'max_len': <Parameter "max_len=80">,
              'text': <Parameter "text:str">})

In [ ]:
# _empty is a special value here since None is quite widely used
# as a valid default value for a function parameter.

In [59]:
sig = signature(tag)

In [60]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 
          'src': 'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag)

In [61]:
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [62]:
for name, value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [63]:
del my_tag['name']

In [64]:
bound_args = sig.bind(**my_tag)

TypeError: missing a required argument: 'name'

In [66]:
# Function annotations


def clip_annot(text: str, max_len: 'int > 0'=80) -> str:
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # No spaces found
        end = len(text)
    return text[:end].rstrip()

In [67]:
clip_annot.__annotations__

{'max_len': 'int > 0', 'return': str, 'text': str}

In [68]:
sig = signature(clip_annot)

In [69]:
sig.return_annotation

str

In [70]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80
